In [ ]:
import numpy as np
import pandas as pd
import datatable as dt

In [ ]:
# load train dataset
X = pd.read_csv('../input/tps-apr-22-fe-v2/train_tps_apr_22_fev2.csv')

y = X[['state']]
groups = X['sequence']
X = X.drop(['sequence', 'subject'], inplace=False, axis=1)

# LightGBMError: Do not support special JSON characters in feature name.
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
X.index

In [ ]:
from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import optuna

def objective(trial, data=X, target=y):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # GroupKFold split once
    # X_train, X_test, y_train, y_test = kf.split(X, y, groups.unique())
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        'n_estimators': trial.suggest_int('n_estimators', 500, 20000, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 0.9, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.2, 0.9, 0.1),
        'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.2, 0.9, 0.1),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 1e4),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 1e4),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 1e4),
        'gamma': trial.suggest_loguniform('gamma', 1e-4, 1e4),
        'predictor': "gpu_predictor",
        'eval_metric': 'auc',
        'objective': 'binary:logistic',    # classification problem
         }
    
    
    
    # kf = GroupKFold(n_splits=3)
    oof = np.zeros(len(X))
    idx1 = X.index
    res = 0
    
    # gkf = GroupKFold(n_splits=3)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=False)
        
    model = xgb.XGBClassifier(**params, tree_method='gpu_hist', random_state=42, use_label_encoder=False)        
    model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], 
                early_stopping_rounds=300, verbose=False)
    
    preds = model.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, preds)
    score = auc(fpr, tpr)
    
    return score

In [ ]:
import xgboost as xgb

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, gc_after_trial=True)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# {'max_depth': 8, 'n_estimators': 3000, 'learning_rate': 0.08530569310433453, 'subsample': 0.2, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.30000000000000004, 'min_child_weight': 0.467428476013027, 'reg_lambda': 0.2279498019218528, 'reg_alpha': 171.24877820234045, 'gamma': 1.2435329218902063}

In [ ]:
# optuna auc == 1.0 ???
# lgbm
# {'n_estimators': 16500, 'learning_rate': 0.06946446628215118, 'reg_alpha': 0.010253222571604808, 'reg_lambda': 0.031961179402523146, 'colsample_bytree': 1.0, 'subsample': 0.6, 'max_depth': 100, 'num_leaves': 867, 'min_child_samples': 291, 'min_data_per_groups': 26}

# xgb
# {'max_depth': 7, 'n_estimators': 18500, 'learning_rate': 0.055391555488858234, 'subsample': 0.8, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.8, 'min_child_weight': 0.026558310584072646, 'reg_lambda': 0.001979996038118179, 'reg_alpha': 6.848333778413986, 'gamma': 0.03131259189530803}